# Read SeaWIFs monthly euphotic depth data

this R notebook reads netcdf files from L3 Browser @ NASA Ocean Color https://oceancolor.gsfc.nasa.gov/l3/

the data is from 8-day running averages of euphotic depth measured by SeaWIFs extracted from the Cariaco Region

In [11]:
library("ncdf4")
library("reshape2")

In [12]:
outputfile <- "DATA_processed/SeaWIFs_EuphoticDepth_extracted.csv"
if (file.exists(outputfile)) file.remove(outputfile)    # caution new data will be appended to this file if it already exists
print(outputfile)

[1] TRUE

[1] "DATA_processed/SeaWIFs_EuphoticDepth_extracted.csv"


## create bounding box & create list of file names

In [13]:
# list and remove objects
ls()
rm(list = ls())

# 10.500000, -64.666667

# set the study area
latmax <- 10.6
latmin <- 10.4
lonmax <- -64.75
lonmin <- -65.6

# create a list of files and indicate its length
f <- list.files("DATA_raw/SeaWIFs_euphoticdepth_monthly/.", pattern="*.L3m_MO_ZLEE.x_Zeu_lee.nc",full.names=F)
lf<-length(f)
print(head(f))

[1] "outputfile"

[1] "S19972741997304.L3m_MO_ZLEE.x_Zeu_lee.nc"
[2] "S19973051997334.L3m_MO_ZLEE.x_Zeu_lee.nc"
[3] "S19973351997365.L3m_MO_ZLEE.x_Zeu_lee.nc"
[4] "S19980011998031.L3m_MO_ZLEE.x_Zeu_lee.nc"
[5] "S19980321998059.L3m_MO_ZLEE.x_Zeu_lee.nc"
[6] "S19980601998090.L3m_MO_ZLEE.x_Zeu_lee.nc"


## show netcdf file structure

In [14]:
data<-nc_open(paste('DATA_raw/SeaWIFs_euphoticdepth_monthly/',f[1],sep=''))
print(data)
# extract data
nc_close(data)

File DATA_raw/SeaWIFs_euphoticdepth_monthly/S19972741997304.L3m_MO_ZLEE.x_Zeu_lee.nc (NC_FORMAT_NETCDF4):

     2 variables (excluding dimension variables):
        short Zeu_lee[lon,lat]   (Chunking: [26,26])  (Compression: shuffle,level 4)
            long_name: Euphotic depth, Lee algorithm
            scale_factor: 0.00499999988824129
            add_offset: 160
            units: m
            _FillValue: -32767
            valid_min: -32000
            valid_max: 4000
            display_scale: linear
            display_min: 5
            display_max: 180
        unsigned byte palette[eightbitcolor,rgb]   (Contiguous storage)  

     4 dimensions:
        lat  Size:75
            long_name: Latitude
            units: degrees_north
            standard_name: latitude
            _FillValue: -999
            valid_min: -90
            valid_max: 90
        lon  Size:75
            long_name: Longitude
            units: degrees_east
            standard_name: longitude
          

## define variable to extract : PAR

In [15]:
# variable
var<-"Zeu_lee"

## Open each individual file and extract PAR values within bounding box defined above

In [16]:
for (i in 1:lf) {
  # progress indicator
  print(paste("Processing file",i,"from",length(f),sep=" "))
  # open netCDF file
  data<-nc_open(paste('DATA_raw/SeaWIFs_euphoticdepth_monthly/',f[i],sep=''))
  # extract data
  lon<-ncvar_get(data,"lon")
  lat<-ncvar_get(data,"lat")
  value<-ncvar_get(data,var)
  unit<-ncatt_get(data,var,"units")$value
  # matrix to data.frame
  dimnames(value)<-list(lon=lon,lat=lat)
  dat.var<-melt(value,id="lon")
  # select data from the study area taking out missing data
  dat.varSAtmp<-subset(dat.var,lon<=lonmax & lon>=lonmin & lat<=latmax & lat>=latmin)
  # extract date information
  dateini<-ncatt_get(data,0,"time_coverage_start")$value
  dateend<-ncatt_get(data,0,"time_coverage_end")$value
  datemean<-mean(c(as.Date(dateend,"%Y-%m-%dT%H:%M:%OSZ"),as.Date(dateini,"%Y-%m-%dT%H:%M:%OSZ")))
  year<- substring(datemean,0,4) #1997 + ((i+6)/12)#
  month<- substring(datemean,6,7) #i+6 #
  # prepare final data set
  dat.varSA<-data.frame(rep(as.integer(year,nrow(dat.varSAtmp))),rep(as.integer(month,nrow(dat.varSAtmp))),
                        dat.varSAtmp,rep(unit,nrow(dat.varSAtmp)),rep(var,nrow(dat.varSAtmp)))
  names(dat.varSA)<-c("year","month","lon","lat","value","unit","var")
  # save csv file
  fe<-file.exists(paste('DATA_processed/','SeaWIFs_EuphoticDepth_extracted.csv',sep=''))
  write.table(dat.varSA,paste('DATA_processed/','SeaWIFs_EuphoticDepth_extracted.csv',sep=''),row.names=FALSE,col.names=!fe,sep=",",dec=".",append=fe)
  # close connection
  nc_close(data)
  # clean workspace
  rm(data,lon,lat,value,unit,dat.var,dat.varSAtmp,dateini,dateend,datemean,year,month,dat.varSA,fe)
}
rm(var,f,i,latmax,latmin,lf,lonmax,lonmin)

[1] "Processing file 1 from 152"
[1] "Processing file 2 from 152"
[1] "Processing file 3 from 152"
[1] "Processing file 4 from 152"
[1] "Processing file 5 from 152"
[1] "Processing file 6 from 152"
[1] "Processing file 7 from 152"
[1] "Processing file 8 from 152"
[1] "Processing file 9 from 152"
[1] "Processing file 10 from 152"
[1] "Processing file 11 from 152"
[1] "Processing file 12 from 152"
[1] "Processing file 13 from 152"
[1] "Processing file 14 from 152"
[1] "Processing file 15 from 152"
[1] "Processing file 16 from 152"
[1] "Processing file 17 from 152"
[1] "Processing file 18 from 152"
[1] "Processing file 19 from 152"
[1] "Processing file 20 from 152"
[1] "Processing file 21 from 152"
[1] "Processing file 22 from 152"
[1] "Processing file 23 from 152"
[1] "Processing file 24 from 152"
[1] "Processing file 25 from 152"
[1] "Processing file 26 from 152"
[1] "Processing file 27 from 152"
[1] "Processing file 28 from 152"
[1] "Processing file 29 from 152"
[1] "Processing file 30